##  Chameleon/FABRIC L2 Stitching

This notebook demonstrates how to stitch Chameleon experiments to FABRIC.

### Set Chameleon Environment

FABRIC should already be configured in your Jupyter environment. If this is your first time using FABRIC, may need to follow the [Configure Environment](./fabric_examples/fablib_api/configure_environment/configure_environment.ipynb) notebook to complete the configuration.

Set the following environment vars to the values found in your custom Chameleon-openrc.sh file. You can obtain your Chameleon-openrc.sh from Chameleon using these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#the-openstack-rc-script).  The expected password is not your regular Chameleon password. Instead, you need to create a Chameleon CLI password by following these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#cli-authentication)

In [ ]:
import os
import sys


module_path = os.path.abspath(os.path.join(f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fablib_local"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from chameleon_utils.chameleon_config import *
load_chameleon_rc_environment(chameleon_rc_file=f"{os.environ['HOME']}/work/fablib_local_private_config/Chameleon-openrc.sh")

    
from fablib_custom.fablib_custom import *
from fablib_common_utils.utils import *
from fablib_common_utils.fabric_fabnet_slice import *

from chameleon_utils.chameleon_stitching import *
from chameleon_utils.chameleon_servers import *

### Import Libraries

In [ ]:
#General imports
import os
import json
import traceback
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
from datetime import datetime, timedelta
from dateutil import tz
import time

# Chameleon Library
import chi
import chi.lease 
from chi.server import *
from chi.lease import *
from chi.network import *

# FABRIC Library
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

In [ ]:
time_stamp = datetime.now(tz=tz.tzutc()).strftime('%Y%m%d%H%M')

ext=f'_pruth_{time_stamp}'

### Chameleon Experiment Variable

In [ ]:
# Chameleon Config
chameleon_prefix =  f"pruth_fabric_stitch{ext}"
chameleon_server_name = chameleon_prefix+'server'
chameleon_network_name = chameleon_prefix+'net'
chameleon_subnet_name = chameleon_prefix+'subnet'
chameleon_router_name = chameleon_prefix+'router'
chameleon_lease_name = chameleon_prefix+'lease'

chameleon_image_name='CC-Ubuntu20.04'
chameleon_node_type="compute_cascadelake_r"
chameleon_physical_network='physnet1'
chameleon_stitch_provider='fabric'
chameleon_server_count=2
chameleon_key_name='my_chameleon_key'

### FABRIC Experiment Variables

In [ ]:
# Create a FABlib manager
fablib = fablib_manager()
fablib.show_config()

# FABRIC Config
fabric_slice_name=f'chameleon_stitch{ext}'
fabric_node_name='node1'

fabric_node_image='default_ubuntu_20'
#fabric_site=fablib.get_random_site()
fabric_site='STAR'
print(f'fabric_site: {fabric_site}')

### Network Variables

The stitched network spans both Chameleon and FABRIC. The network needs a common subnet but distinct IP address allocation pools.

In [ ]:
#Network Config
subnet = IPv4Network("192.168.100.0/24")

fabric_allocation_pool_start=IPv4Address('192.168.100.200')
fabric_allocation_pool_end=IPv4Address('192.168.100.250')
fabric_available_ips=[]
for ip_int in range(int(fabric_allocation_pool_start),int(fabric_allocation_pool_end)+1):
    fabric_available_ips.append(IPv4Address(ip_int))
    
chameleon_allocation_pool_start='192.168.100.100'
chameleon_allocation_pool_end='192.168.100.150'
chameleon_gateway_ip='192.168.100.1'

## Create the Chameleon Network

#### Create a Lease at Chicago

In [ ]:
BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
#chi.lease.add_node_reservation(reservation_list, count=chameleon_server_count, node_type=chameleon_node_type)
#chi.lease.add_fip_reservation(reservation_list, count=2)

reservation_list.append(
        {
            "resource_type": "network",
            "network_name": chameleon_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", chameleon_stitch_provider]
            ),
        }
)

# Create the lease
chameleon_lease = chi.lease.create_lease(chameleon_lease_name,
                                  reservations=reservation_list,
                                  start_date=start_date,
                                  end_date=end_date)
    
#Print the lease info
#print(json.dumps(chameleon_lease, indent=2))
#chameleon_compute_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
chameleon_network_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
#chameleon_floatingip_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

#print(f"chameleon_compute_reservation_id: {chameleon_compute_reservation_id}")
print(f"chameleon_network_reservation_id: {chameleon_network_reservation_id}")
#print(f"chameleon_floatingip_reservation_id: {chameleon_floatingip_reservation_id}")

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
network_vlan = None
while network_vlan == None:
    try:
        #Get the network
        chameleon_network = chi.network.get_network(chameleon_network_name)

        #Get the network ID
        chameleon_network_id = chameleon_network['id']
        print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        network_vlan = chameleon_network['provider:segmentation_id']
        print(f'network_vlan: {network_vlan}')
    except:
        print(f'Chameleon Network is not ready. Trying again!')
        time.sleep(10)           

#### Add a subnet to the reserved network

In [ ]:
chameleon_subnet = chi.network.create_subnet(chameleon_subnet_name, chameleon_network_id, 
                                             cidr=str(subnet),
                                             allocation_pool_start=chameleon_allocation_pool_start,
                                             allocation_pool_end=chameleon_allocation_pool_end,
                                             gateway_ip=chameleon_gateway_ip)

print(json.dumps(chameleon_subnet, indent=2))

#### Add a Router

In [ ]:
chameleon_router = chi.network.create_router(chameleon_router_name, gw_network_name='public')

print(json.dumps(chameleon_router, indent=2))

#### Attach the Router and Subnet

In [ ]:
chi.network.add_subnet_to_router_by_name(chameleon_router_name, chameleon_subnet_name)

## Create FABRIC Node and Link

In [ ]:
try:
    #network_vlan = str(3307)
    
    #Create a slice
    fabric_slice = fablib.new_slice(name=fabric_slice_name)
    
    fabric_node = fabric_slice.add_node(name=fabric_node_name, site=fabric_site, image=fabric_node_image,
                                       cores=2, ram=8, disk=10)
    #fabric_node.add_component(model='NVME_P4510', name='nvme1')

    fabric_node_iface = fabric_node.add_component(model='NIC_Basic', name=f"nicCham").get_interfaces()[0]
    #fabric_node_fabnet_iface = fabric_node.add_component(model='NIC_Basic', name=f"nicFabnet").get_interfaces()[0]
    #[fabric_node_iface,fabric_node_fabnet_iface] =  fabric_node.add_component(model='NIC_ConnectX_5', name=f"nic1").get_interfaces()


    fabric_facility_port = fabric_slice.add_facility_port(name='Chameleon-StarLight', site='STAR', vlan=str(network_vlan))
    fabric_facility_port_iface = fabric_facility_port.get_interfaces()[0]

    fabric_net = fabric_slice.add_l2network(name=f'net_facility_port', interfaces=[fabric_node_iface,fabric_facility_port_iface]) 
    
    #fabric_fabnet = fabric_slice.add_l3network(name=f'net_fabnet', interfaces=[fabric_node_fabnet_iface], type='IPv4')


    #Submit the Request
    fabric_slice.submit()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:
try:
    fabric_slice = fablib.get_slice(fabric_slice_name)
    #fabric_slice.wait_jupyter()
except Exception as e:
    print(f"Exception: {e}")

## Configure the FABRIC VM's IP Address

In [ ]:
try:        
    fabric_node = fabric_slice.get_node(name=fabric_node_name)   
    
    fabric_node_iface = fabric_node.get_interface(network_name=f'net_facility_port') 
    fabric_node_addr = fabric_available_ips.pop(0)
    print(f"fabric_node_addr: {fabric_node_addr}")
    fabric_node_iface.ip_addr_add(addr=fabric_node_addr, subnet=subnet)
    
    stdout, stderr = fabric_node.execute(f'ip addr show {fabric_node_iface.get_os_interface()}')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    net_fabnet = fabric_slice.get_network(name=f'net_facility_port')
    net_fabnet_available_ips = net_fabnet.get_available_ips()
    print(f"{net_fabnet}")
    
    fabric_node = fabric_slice.get_node(name=fabric_node_name)        
    fabric_node_iface = fabric_node.get_interface(network_name=f'net_facility_port')  
    fabric_node_addr = net_fabnet_available_ips.pop(0)
    fabric_node_iface.ip_addr_add(addr=fabric_node_addr, subnet=net_fabnet.get_subnet())
    
    #node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    
    #stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    #print (stdout)
    
    #stdout, stderr = node1.execute(f'ip route list')
    #print (stdout)
except Exception as e:
    print(f"Exception: {e}")

## (Optional) Test the Link

We can test the facility port link by pinging the Chameleon gateway IP from the FABRIC VM.

In [ ]:
try:
    fabric_node = fabric_slice.get_node(name=fabric_node_name)     
    fabric_node_iface = fabric_node.get_interface(network_name=f'net_facility_port') 

    stdout, stderr = fabric_node.execute(f'ping -c 5  {chameleon_gateway_ip}')

    #stdout, stderr = fabric_node.execute(f'ping -c 5 192.168.100.106')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

## Start Chameleon Servers

In [ ]:
try:
    fabric_node = fabric_slice.get_node(name=fabric_node_name)   
    
    #nvme1 = fabric_node.get_component('nvme1')
    #nvme1.configure_nvme()

    
    stdout, stderr = fabric_node.execute('echo Hello, FABRIC from node `hostname -s`')
    print(stdout)

    stdout, stderr = fabric_node.execute('sudo apt-get -y -q update && sudo apt-get -y -q install python3 python3-pip swift python3-openstackclient apache2')
    print(stdout)

    stdout, stderr = fabric_node.execute('pip3 install python-swiftclient ')
    print(stdout)

    fabric_node.upload_file('swift_get.sh','swift_get.sh')
    fabric_node.upload_file('swift_put.sh','swift_put.sh')
    fabric_node.upload_file('swift_list.sh','swift_list.sh')
    fabric_node.upload_file('CH-822154-openrc.sh','CH-822154-openrc.sh')
    
    fabric_node.ip_route_add(subnet=IPv4Network("192.5.87.2/32") , gateway=IPv4Address(chameleon_gateway_ip))
    
    #10.129.128.0/17 via  10.133.1.1  (Router to all other FABnet subnets
    fabric_node.ip_route_add(subnet=IPv4Network("10.128.0.0/12") , gateway=net_fabnet.get_gateway())

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    
    stdout,stderr = fabric_node.execute('sudo systemctl start apache2')
    print(stdout)
    print(stderr)    
    
    stdout,stderr = fabric_node.execute('chmod +x *.sh && export PATH=$PATH:/home/ubuntu/.local/bin && source CH-822154-openrc.sh && ./swift_list.sh')
    print(stdout)
    print(stderr)
    
    #openstack container save pruth-container

    #stdout,stderr = fabric_node.execute('sudo chown -R ubuntu /var/www && chmod +x *.sh && export PATH=$PATH:/home/ubuntu/.local/bin && source CH-822154-openrc.sh && cd /var/www/html && ~/swift_get.sh pruth-container files/file.25G')
    #print(stdout)
    #print(stderr)    
    
   
except Exception as e:
    print(f"Exception: {e}")    
    

In [ ]:
#import chi.server
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    # Create the server
    server = chi.server.create_server(server_name, 
                                  reservation_id=chameleon_compute_reservation_id, 
                                  network_name=chameleon_network_name, 
                                  image_name=chameleon_image_name,
                                  key_name=chameleon_key_name
                                 )
# Wait until the server is active
#chi.server.wait_for_active(server.id)

## Get the Chameleon Server Fixed IPs

In [ ]:
#get fixed ips
fixed_ips={}
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    server_id = get_server_id(server_name)
    fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    fixed_ips[server_name]=fixed_ip

for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')

## Test the Link by Pinging All Chameleon Nodes

In [ ]:
for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')
    
    stdout, stderr = fabric_node.execute(f'ping -c 5 {fixed_ip}')
    print (stdout)
    print (stderr)

# Clean Up 

## Delete Chameleon Resources

Delete the servers

In [ ]:
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    chi.server.delete_server(get_server_id(server_name))

#### De-configure Network

In [ ]:
router_id = chameleon_router['id']
subnet_id = chameleon_subnet['id']
network_id = chameleon_network_id

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass

## Release Lease

In [ ]:
chi.lease.delete_lease(chameleon_lease['id'])

## Delete FABRIC Slice

In [ ]:
try:
    fabric_slice = fablib.get_slice(fabric_slice_name)
    fabric_slice.delete()
except Exception as e:
    print(f"Exception: {e}")